In [2]:
import qe.myqengine as qe
import pandas as pd
import numpy as np

In [3]:
time_series = qe.TimeSeries(start_date = '0M',end_date = '0M', frequency='M',calendar='NAY')

univ = qe.IdUniverse(ids = ['AAPL-US','META-US','AMZN-US','NVDA-US','GOOGL-US','MSFT-US','TSLA-US'],universe_type='Equity')
req = qe.calculate(universe = univ, dates = time_series,formulas = ["FE_BROKER_ESTIMATE(SNAP,EPS,BKR_NAME,QTR_ROLL,+1,#DATE,,',AWIN=1D,ESTDATE=INPUT,WIN=31,')",
                                            "FE_BROKER_ESTIMATE(SNAP,EPS,AN_NAME,QTR_ROLL,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT,')",
                                           "FE_BROKER_ESTIMATE(SNAP,EPS,EST_VALUE,QTR_ROLL,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT')",
                                           "FE_BROKER_ESTIMATE(SNAP,EPS,PREV_VALUE,QTR_ROLL,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT')",
                                        "FE_BROKER_ESTIMATE_DATE(SNAP,EPS,INPUTDATETIME,QTR_ROLL,+1,'YYYYMMDD',#DATE,,'AWIN=1D,ESTDATE=INPUT,WIN=31')",
                                        "FE_BROKER_ESTIMATE(SNAP,EPS,AN_ENTITY_ID,QTR_ROLL,+1,#DATE,,',AWIN=1D,ESTDATE=INPUT,WIN=31,')",
                                        "FE_ESTIMATE_DATE(RPT_DATE,,QTR_ROLL,+1,'YYYYMMDD',#DATE,,,'')",
                                        "FE_SURPRISE(PERCENT,EPS,MEAN,QTR_ROLL,0,#DATE,,,'')",
                                        "P_PRICE_RETURNS(1,20220831,#DATE,RANGE,USD)"],
                                        source = 'FqlExpression',
                                        is_array_return_type=True)

df = req.data.rename(columns={'UNIVERSE':"Ticker"}).set_index("Ticker")
df= df.rename(columns = {"FE_BROKER_ESTIMATE(SNAP,EPS,BKR_NAME,QTR_ROLL,+1,#DATE,,',AWIN=1D,ESTDATE=INPUT,WIN=31,')":"Broker Name",
                     "FE_BROKER_ESTIMATE(SNAP,EPS,AN_NAME,QTR_ROLL,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT,')":'Analyst Name',
                    "FE_BROKER_ESTIMATE(SNAP,EPS,EST_VALUE,QTR_ROLL,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT')":"EST_EPS_FY1",
                    "FE_BROKER_ESTIMATE(SNAP,EPS,PREV_VALUE,QTR_ROLL,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT')":"PREV_EST_EPS_FY1",
                    "FE_BROKER_ESTIMATE_DATE(SNAP,EPS,INPUTDATETIME,QTR_ROLL,+1,'YYYYMMDD',#DATE,,'AWIN=1D,ESTDATE=INPUT,WIN=31')":'Research Date',
                    "FE_BROKER_ESTIMATE(SNAP,EPS,AN_ENTITY_ID,QTR_ROLL,+1,#DATE,,',AWIN=1D,ESTDATE=INPUT,WIN=31,')":"Analyst ID",
                    "P_PRICE_RETURNS(1,20220831,#DATE,RANGE,USD)":"Cumulative Return",
                    "FE_ESTIMATE_DATE(RPT_DATE,,QTR_ROLL,+1,'YYYYMMDD',#DATE,,,'')":'Report Date',
                    "FE_SURPRISE(PERCENT,EPS,MEAN,QTR_ROLL,0,#DATE,,,'')":"Surprise",
                    "DATE":'Date'
                    })

df['Date'] = pd.to_datetime(df['Date']).apply(lambda x:x.strftime("%m/%d/%Y"))
df['Cumulative Return']= df['Cumulative Return'].apply(lambda x:x[0])
df['Report Date']= pd.to_datetime(df['Report Date'].apply(lambda x: x[0]))
df['Surprise']= df['Surprise'].apply(lambda x: x[0]).astype(float)
df_e = pd.concat([df['Analyst ID'].explode().reset_index()[['Ticker','Analyst ID']],df['Broker Name'].explode().reset_index().iloc[:,-1:],df['Analyst Name'].explode().reset_index().iloc[:,-1:],df['EST_EPS_FY1'].explode().reset_index().iloc[:,-1:],df['PREV_EST_EPS_FY1'].explode().reset_index().iloc[:,-1:],df['Research Date'].explode().reset_index().iloc[:,-1:]],axis=1)
df_e['Research Date'] = pd.to_datetime(df_e['Research Date'])
df_e = df_e.replace('Restricted',np.nan).replace('@NA',np.nan)
df_e = df_e.groupby(['Research Date','Broker Name','Ticker']).agg({'EST_EPS_FY1':'mean'})

df_e.head()

EST_EPS_FY1
Research Date Broker Name               Ticker              
2023-08-02    KeyBanc Capital Markets   MSFT-US     2.650000
              Needham                   META-US     3.050000
2023-08-03    Credit Suisse             AAPL-US     1.396731
              Daiwa Securities Co. Ltd. META-US     3.650000
              Jefferies                 AMZN-US     0.530000